<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/mask_detection_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the 'mlflow' library for managing the machine learning lifecycle
!pip install mlflow
# Install the 'opendatasets' library to load online datasets
!pip install opendatasets
# Import necessary libraries
import mlflow
from mlflow import keras
import mlflow.keras
import opendatasets as od
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
import pickle
#Load the dataset
od.download("https://www.kaggle.com/datasets/prithwirajmitra/covid-face-mask-detection-dataset")
# Create a new MLflow experiment named "first_experiment"
experiment = mlflow.create_experiment("first_experiment")
# Set the active MLflow experiment to "first_experiment"
mlflow.set_experiment('first_experiment')

# Initialize the ImageDataGenerator with data augmentation and preprocessing settings
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Rescale pixel values to the range [0, 1]
    rotation_range=20,  # Randomly rotate images in the range (-20, 20) degrees
    width_shift_range=0.1,  # Randomly shift the width by up to 10% of the image width
    height_shift_range=0.1,  # Randomly shift the height by up to 10% of the image height
    shear_range=0.2,  # Shear transformation with a maximum shear angle of 20 degrees
    zoom_range=0.2,  # Randomly zoom in or out by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest',  # Fill empty pixels with the nearest available pixel value
)

# Create data iterators for training, validation, and testing sets
train_iterator = datagen.flow_from_directory(
    "covid-face-mask-detection-dataset/New Masks Dataset/Train",
    class_mode='binary',
    batch_size=30,
    shuffle=True
)

valid_iterator = datagen.flow_from_directory(
    "covid-face-mask-detection-dataset/New Masks Dataset/Validation",
    class_mode='binary',
    batch_size=30,
    shuffle=True
)

test_iterator = datagen.flow_from_directory(
    "covid-face-mask-detection-dataset/New Masks Dataset/Test",
    class_mode='binary',
    batch_size=30,
    shuffle=True
)

# Define a function to create a CNN model
def get_model(params):
    # Initialize a Sequential model
    model = Sequential()

    # Add the first convolutional layer with specified parameters
    model.add(Conv2D(params['nb_filters'], params['filter_size'], padding='same', activation='relu', input_shape=(256, 256, 3)))

    # Add a second convolutional layer with the same parameters
    model.add(Conv2D(params['nb_filters'], params['filter_size'], padding='same', activation='relu'))

    # Add a max-pooling layer
    model.add(MaxPooling2D())

    # Flatten the output for the fully connected layers
    model.add(Flatten())

    # Add a dense layer with specified number of neurons and ReLU activation
    model.add(Dense(params['nb_neurons'], activation='relu'))

    # Add the output layer with a single neuron and sigmoid activation for binary classification
    model.add(Dense(1, activation='sigmoid'))

    return model
# Define a function to train a machine learning model
def train_model(params, text_params, train_iterator, valid_iterator):
    # Create a model based on the specified parameters
    model = get_model(params)

    # Compile the model with the specified optimizer, loss function, and metrics
    model.compile(optimizer=text_params['optimizer'], loss='binary_crossentropy', metrics=text_params['metrics'])

    # Train the model using the training data and validate it using the validation data
    history = model.fit(train_iterator, validation_data=valid_iterator, epochs=params['epochs'])

    return history, model

# Define a function to log experiment information and model using MLflow
def mlflow_log(params, history, text_params, model, run_num):
    # Extract the metrics from the training history
    metrics = history.history

    # Get the last value for each metric
    for key in metrics.keys():
        metrics[key] = metrics[key][-1]

    # Start a new MLflow run for the experiment
    with mlflow.start_run() as run:
        # Log experiment parameters
        mlflow.log_params(params)

        # Log the metrics from the training history
        mlflow.log_metrics(metrics)

        # Log textual parameters
        for text_item in text_params.items():
            if isinstance(text_item[1], list):
                for metric_name in text_item[1]:
                    mlflow.log_text(metric_name, text_item[0] + '.txt')
            else:
                mlflow.log_text(text_item[1], text_item[0] + '.txt')

        # Log the Keras model as an artifact
        mlflow.keras.log_model(model, artifact_path=f"model_{run_num}")
# Set multiple runs,1st run:
params = {
    'filter_size':4,
    'nb_filters':4,
    'nb_neurons':70,
    'epochs':4,
    'batch_size':20
}

text_params = {
    'optimizer':'sgd',
    'metrics':['accuracy']
}

run_num = 1

history,model=train_model(params,text_params,train_iterator,valid_iterator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model,run_num)
# 2nd run
params = {
    'filter_size':3,
    'nb_filters':3,
    'nb_neurons':100,
    'epochs':3,
    'batch_size':20
}

text_params = {
    'optimizer':'sgd',
    'metrics':['accuracy']
}

run_num = 2

history,model=train_model(params,text_params,train_iterator,valid_iterator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model,run_num)
# 3rd run
params = {
    'filter_size':3,
    'nb_filters':3,
    'nb_neurons':70,
    'epochs':3,
    'batch_size':20
}

text_params = {
    'optimizer':'adam',
    'metrics':['accuracy']
}

run_num =3

history,model=train_model(params,text_params,train_iterator,valid_iterator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model,run_num)
# 4th run
params = {
    'filter_size':5,
    'nb_filters':5,
    'nb_neurons':100,
    'epochs':6,
    'batch_size':20
}

text_params = {
    'optimizer':'sgd',
    'metrics':['accuracy']
}

run_num =4

history,model=train_model(params,text_params,train_iterator,valid_iterator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model,run_num)
# 5th run
params = {
    'filter_size':3,
    'nb_filters':3,
    'nb_neurons':110,
    'epochs':8,
    'batch_size':20
}

text_params = {
    'optimizer':'sgd',
    'metrics':['accuracy']
}

run_num = 5

history,model=train_model(params,text_params,train_iterator,valid_iterator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model,run_num)
#6th run
params = {
    'filter_size':4,
    'nb_filters':3,
    'nb_neurons':110,
    'epochs':10,
    'batch_size':20
}

text_params = {
    'optimizer':'sgd',
    'metrics':['accuracy']
}

run_num = 6

history,model=train_model(params,text_params,train_iterator,valid_iterator)

print("[INFO] Logging the parameters, textual info & best values for each metric & the model...")
mlflow_log(params, history, text_params, model,run_num)
# Search for runs within the specified experiment
mlflow.search_runs(experiment)
# Define a final model based on the best parameters obtained from the experiment
def final_model():
  model=Sequential()
  model.add(Conv2D(3,(3,3),padding='same',activation='relu',input_shape=(256,256,3)))
  model.add(Conv2D(3,(3,3),padding='same',activation='relu'))
  model.add(MaxPooling2D())
  model.add(Flatten())
  model.add(Dense(110,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer="sgd",loss="binary_crossentropy",metrics=['acc'])
  return model
#Train and evaluate the model
model=final_model()
model.fit(train_iterator,validation_data=valid_iterator,epochs=8,batch_size=20,verbose=0)
model.evaluate(test_iterator)
# Save the model to a file using pickle.dump()
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)